# Исследование надёжности заёмщиков
## Описание проекта

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Итоги проекта
На основе данных кредитного отдела банка исследовали влияние семейного положения и количества детей на факт погашения кредита в срок. Была получена информация о данных. Определены и обработаны пропуски. Заменены типы данных на соответствующие
хранящимся данным. Удалены дубликаты. Выделены леммы в значениях столбца и категоризированны данные.

## Содержание
1. [Изучение общей информации](#header_1)
2. [Предобработка данных](#header_2)

    - [Общая инфорамация](#header_2_1)
    - [Обработка пропусков](#header_2_2)
    - [Обработка пропусков: выводы](#header_2_3)
    - [Замена типов данных](#header_2_4)
    - [Замена типов данных: выводы](#header_2_5)
    - [Обработка дубликатов](#header_2_6)
    - [Обработка дубликатов: выводы](#header_2_7)
    - [Лемматизация](#header_2_8)
    - [Лемматизация: выводы](#header_2_9)
    - [Категоризация данных](#header_2_10)
    - [Категоризация данных: выводы](#header_2_11)
    
    
3. [Анализ данных](#header_3)

    - [Зависимость между наличием детей и возвратом кредита в срок](#header_3_1)
    - [Зависимость между семейным положением и возвратом кредита в срок](#header_3_2)
    - [Зависимость между уровнем дохода и возвратом кредита в срок](#header_3_3)
    - [Влияние цели кредита на возврат в срок](#header_3_4)
    
    
4. [Общий вывод](#header_4)



# Изучение общей информации <a id='header_1'></a>

In [16]:
import pandas as pd
data = pd.read_csv('datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [17]:
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


In [18]:
data.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

### Вывод

В файле содержится 21 525 записей. Названия полей унифицированы. Типы данных по смыслу соответсвтуют полям. Не везде регистр строк в текстовых полях одинаковый для одинаковых по смыслу значений.

Для полей days_employed (общий трудовой стаж в днях), total_income (ежемесячный доход) имеются пропущенные значения. 

Для поля days_employed (общий трудовой стаж в днях) существуют отрицательные значения, а также очень большие значения — это выглядит странно. 

В данных есть словари: образование и семейное положение заданы и как строки, и как id.

Цели кредитования очень разные.

# Предобработка данных <a id='header_2'></a>

## Общая информация <a id='header_2_1'></a>

Уже понимаем, что есть пропуски в числовых полях. Посмотрим, что происходит в строковых:

In [19]:
display(data.groupby('children')['children'].count().to_frame())
display(data['education'].value_counts().to_frame())
display(data['gender'].value_counts().to_frame())
display(data['income_type'].value_counts().to_frame())
display(data['purpose'].value_counts().to_frame())
display(data.groupby('dob_years')['dob_years'].count().to_frame())


,children
children,
-1,47
0,14149
1,4818
2,2055
3,330
4,41
5,9
20,76


,education
среднее,13750
высшее,4718
СРЕДНЕЕ,772
Среднее,711
неоконченное высшее,668
ВЫСШЕЕ,274
Высшее,268
начальное,250
Неоконченное высшее,47
НЕОКОНЧЕННОЕ ВЫСШЕЕ,29


,gender
F,14236
M,7288
XNA,1


,income_type
сотрудник,11119
компаньон,5085
пенсионер,3856
госслужащий,1459
предприниматель,2
безработный,2
студент,1
в декрете,1


,purpose
свадьба,797
на проведение свадьбы,777
сыграть свадьбу,774
операции с недвижимостью,676
покупка коммерческой недвижимости,664
операции с жильем,653
покупка жилья для сдачи,653
операции с коммерческой недвижимостью,651
жилье,647
покупка жилья,647


,dob_years
dob_years,
0,101
19,14
20,51
21,111
22,183
23,254
24,264
25,357
26,408


Есть странные значения для детей и возраста (подробнее описано в выводах к шагу). 

Для поля `education` нужно привести значения к единому виду.

In [20]:
#приведем к нижнему регистру и проверим
data['education'] = data['education'].str.lower()
data['education'].value_counts().to_frame()

,education
среднее,15233
высшее,5260
неоконченное высшее,744
начальное,282
ученая степень,6


Пока не начали избавляться от NaN значений, посмотрим дубли:

In [21]:
print('Количество дубликатов до:', data.duplicated().sum())

Количество дубликатов до: 71


In [22]:
#просто чтобы посмотреть и оценить глазами
data[data.duplicated(keep = False)].sort_values(by = data.columns.values.tolist())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
15892,0,NaN,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,NaN,сделка с подержанным автомобилем
19321,0,NaN,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,NaN,сделка с подержанным автомобилем
3452,0,NaN,29,высшее,0,женат / замужем,0,M,сотрудник,0,NaN,покупка жилой недвижимости
18328,0,NaN,29,высшее,0,женат / замужем,0,M,сотрудник,0,NaN,покупка жилой недвижимости
4216,0,NaN,30,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,строительство жилой недвижимости
...,...,...,...,...,...,...,...,...,...,...,...,...
9238,2,NaN,34,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,покупка жилья для сдачи
9013,2,NaN,36,высшее,0,женат / замужем,0,F,госслужащий,0,NaN,получение образования
14432,2,NaN,36,высшее,0,женат / замужем,0,F,госслужащий,0,NaN,получение образования
11033,2,NaN,39,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,сыграть свадьбу


In [23]:
#а что будет, если учитывать только строки, где есть пропущенные значения?
data.loc[(data.duplicated(keep = False)) & (data['days_employed'].isna()) & (data['total_income'].isna())].sort_values(by = data.columns.values.tolist())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
15892,0,NaN,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,NaN,сделка с подержанным автомобилем
19321,0,NaN,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,NaN,сделка с подержанным автомобилем
3452,0,NaN,29,высшее,0,женат / замужем,0,M,сотрудник,0,NaN,покупка жилой недвижимости
18328,0,NaN,29,высшее,0,женат / замужем,0,M,сотрудник,0,NaN,покупка жилой недвижимости
4216,0,NaN,30,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,строительство жилой недвижимости
...,...,...,...,...,...,...,...,...,...,...,...,...
9238,2,NaN,34,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,покупка жилья для сдачи
9013,2,NaN,36,высшее,0,женат / замужем,0,F,госслужащий,0,NaN,получение образования
14432,2,NaN,36,высшее,0,женат / замужем,0,F,госслужащий,0,NaN,получение образования
11033,2,NaN,39,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,сыграть свадьбу


Количество строк сопало с выводом, где мы не накладывали условия на наличие NaN значений. Значит, все дубли именно там, где есть NaN'ы. 

## Обработка пропусков <a id='header_2_2'></a>
### Пропуски `days_employed`
Для задачи проекта поле не является необходимым: заполнять пропуски в нем нет необходимости — можно его исключить из рассмотрения.

In [24]:
#определим количесвто пропусков
print('Пропуски до:', data['days_employed'].isna().sum())

Пропуски до: 2174


In [25]:
#определим все возможные значения для типа занятости
data['income_type'].value_counts().to_frame()

,income_type
сотрудник,11119
компаньон,5085
пенсионер,3856
госслужащий,1459
предприниматель,2
безработный,2
студент,1
в декрете,1


In [26]:
#оценим, что хранится в поле days_employed
print('Отрицательные значения:')
display(data.loc[data['days_employed'] < 0].groupby('income_type')['income_type'].count().to_frame())

print('Положительные значения:')
display(data.loc[data['days_employed']  > 0].groupby('income_type')['income_type'].count().to_frame())

print('Потеряшки:')
display(data.loc[data['days_employed'].isna()].groupby('income_type')['income_type'].count().to_frame())

Отрицательные значения:


,income_type
income_type,
в декрете,1
госслужащий,1312
компаньон,4577
предприниматель,1
сотрудник,10014
студент,1


Положительные значения:


,income_type
income_type,
безработный,2
пенсионер,3443


Потеряшки:


,income_type
income_type,
госслужащий,147
компаньон,508
пенсионер,413
предприниматель,1
сотрудник,1105


In [27]:
#посмотрим на средние значения
print('Отрицателные значения:')
display(data.loc[data['days_employed']  < 0].groupby('income_type')['days_employed'].mean().to_frame())

print('Положительные значения:')
display(data.loc[data['days_employed']  > 0].groupby('income_type')['days_employed'].mean().to_frame())

Отрицателные значения:


,days_employed
income_type,
в декрете,-3296.759962
госслужащий,-3399.896902
компаньон,-2111.524398
предприниматель,-520.848083
сотрудник,-2326.499216
студент,-578.751554


Положительные значения:


,days_employed
income_type,
безработный,366413.652744
пенсионер,365003.491245


В данных пропущено 2174 значения — это 10% от данных. 

Явной связи между этим полем и другими полями таблицы нет — его нельзя вычислить, используя имеющиеся данные. Поэтому для заполнения стоит использовать среднее значение или медиану по полю. При этом нужно разделить клиентов по возрасту и/или типу занятости и определять средние значения внутри групп.

Кроме того, есть подозрительные значения:

- отрицательные для типов занятости: 'в декрете', 'госслужащий', 'компаньон', 'предприниматель', 'сотрудник', 'студент'
- слишком большие (~1000 лет), для типов занятости: 'безработный', 'пенсионер'.

Вероятно, это технические ошибки расчета или ошибки из-за выгрузки из разных источников. Возможно также, что это ошибки ручного ввода: некорректно введены даты начала и окончания трудового стажа.

Предполагаем, что: 
- отрицательные значения возникли из-за того, что перепутаны даты начала и окончания трудового стажа. 
- слишком большие значения возникли из-за того, что учет сделан в часах, а не в днях

In [28]:
def age_group(age):
    '''
    Функция создает категории в зависимости возраста. 
    :param age:  Возраст.
    '''
    if age <= 18:
        return 1
    if 18 < age <= 25: 
        return 2
    if 25 < age <= 35:
        return 3
    if 35 < age <= 45:
        return 4
    if 45 < age <= 60:
        return 5
    return 6

def corr_days_employed (number):
    '''
    Функция корректирует поле количество рабочих дней. 
    Если количество рабочих дней отрицательное, то возврващет абсолютное значение.
    Если положительное, то производится перевод "из часов в дни".
    Если число не определено (NaN) — возвращает исходное значение.
    :param number:  Количество рабочих дней
    '''
    if number < 0:
        return (-1) * number
    if number > 0:
        return number / 24
    return number

#добавляем в датафрейм расчетные поля
data['age_group'] = data['dob_years'].apply(age_group)
data['days_employed_corr'] = data['days_employed'].apply(corr_days_employed)

#для каждой группы возраст-тип_занятости рассчитаем среднее значение рабочих дней
for age_group in data['age_group'].unique():
    for income_type in data['income_type'].unique():
        mean = data.loc[(data['age_group'] == age_group) & (data['income_type'] == income_type), 'days_employed_corr'].mean()
        data.loc[(data['days_employed_corr'].isna()) & (data['age_group'] == age_group) & (data['income_type'] == income_type), 'days_employed_corr'] = mean
        #print(age_group, income_type, mean)

#переприсваиваем, убиваем ненужное поле
data['days_employed'] = data['days_employed_corr']
data.drop('days_employed_corr', axis = 1, inplace = True)


In [29]:
print('Пропуски после:', data['days_employed'].isna().sum())

Пропуски после: 1


In [30]:
#оценим, что хранится в поле days_employed после преобразований
print('Отрицательные значения:')
display(data.loc[data['days_employed'] < 0].groupby('income_type')['income_type'].count().to_frame())

print('Положительные значения:')
display(data.loc[data['days_employed']  > 0].groupby('income_type')['income_type'].count().to_frame())

print('Потеряшки:')
display(data.loc[data['days_employed'].isna()].groupby('income_type')['income_type'].count().to_frame())

Отрицательные значения:


,income_type
income_type,


Положительные значения:


,income_type
income_type,
безработный,2
в декрете,1
госслужащий,1459
компаньон,5085
пенсионер,3856
предприниматель,1
сотрудник,11119
студент,1


Потеряшки:


,income_type
income_type,
предприниматель,1


In [31]:
#поймем, что не так с этим предпринимателем-потеряшкой
data.loc[data['income_type'] == 'предприниматель']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
5936,0,NaN,58,высшее,0,женат / замужем,0,M,предприниматель,0,NaN,покупка жилой недвижимости,5
18697,0,520.848083,27,высшее,0,гражданский брак,1,F,предприниматель,0,499163.144947,на проведение свадьбы,3


Этому предпринимателю ужасно не позвезло. Он оказался один в своей категории возраст-тип_занятости. И к тому же у него изначально пропущено количество рабочих дней. Можно: 
- присвоить ему среднее значение по категории "предприниматель", что не очень верно, так как по другим показателям разница между двумя предпринимателями в данных велика;
- присвоить ему среднее значение по возрастной группе;
- удалить строчку для экономии трудозатрат: этот пользователь представляет совсем незначительную часть данных.

In [32]:
#присвоим среднее значение по возрастной группе и проверим, что все ок
data.loc[(data['age_group'] == 5) & (data['days_employed'].isna()), 'days_employed'] = data.loc[data['age_group'] == 5, 'days_employed'].mean()
data.loc[data['income_type'] == 'предприниматель']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
5936,0,6639.757404,58,высшее,0,женат / замужем,0,M,предприниматель,0,NaN,покупка жилой недвижимости,5
18697,0,520.848083,27,высшее,0,гражданский брак,1,F,предприниматель,0,499163.144947,на проведение свадьбы,3


In [33]:
#посмотрим на средние значения после преобразований
print('Отрцателные значения:')
display(data.loc[data['days_employed']  < 0].groupby('income_type')['days_employed'].mean().to_frame())
print('Положительные значения:')
display(data.loc[data['days_employed']  > 0].groupby('income_type')['days_employed'].mean().to_frame())

Отрцателные значения:


,days_employed
income_type,


Положительные значения:


,days_employed
income_type,
безработный,15267.235531
в декрете,3296.759962
госслужащий,3405.065348
компаньон,2113.217602
пенсионер,15208.080808
предприниматель,3580.302744
сотрудник,2328.511036
студент,578.751554


In [34]:
#еще раз проверим пропуски
print('Пропуски после:', data['days_employed'].isna().sum())

Пропуски после: 0


In [35]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
age_group           21525 non-null int64
dtypes: float64(2), int64(6), object(5)
memory usage: 2.1+ MB


#### Пропуски `total_income`
Для задачи проекта поле не является необходимым. Поэтому заполнять пропуски в нем нет необходимости — можно было бы исключить из рассмотрения.

Явной связи между этим полем и другими полями таблицы нет — его нельзя вычислить, используя имеющиеся данные. Поэтому для заполнения стоит использовать среднее значение или медиану по полю. При этом нужно разделить клиентов по возрасту и/или типу занятости и определять средние значения внутри групп.

In [36]:
#определим количесвто пропусков
print('Пропуски до:', data['total_income'].isna().sum())

Пропуски до: 2174


In [37]:
#изучим средние значения для дохода в разрезе категорий возраста и типа занятости 
#print(data.groupby(['income_type', 'age_group'])['total_income'].mean())

#для разнообразия сводная таблица
data.pivot_table(index = ['income_type', 'age_group'], values = 'total_income', aggfunc = 'mean')

total_income
income_type     age_group               
безработный     3           59956.991984
                4          202722.511368
в декрете       4           53829.130729
госслужащий     1          146973.284116
                2          153193.589071
                3          171158.610012
                4          172401.746570
                5          170258.950923
                6          188917.891473
компаньон       1          185695.921451
                2          160833.004712
                3          199092.929487
                4          212044.462929
                5          206745.408362
                6          203126.613347
пенсионер       1          138492.966270
                2           89368.600062
                3          120001.977997
                4          158029.877544
                5          139408.396395
                6          133610.453466
предприниматель 3          499163.144947
сотрудник       1          156600.145048
                2          139052.559066
                3          160816.375494
                4          165685.826513
                5          161812.489952
                6          173412.775461
студент         2           98201.625314

In [38]:
#для каждой группы возраст-тип_занятости рассчитаем среднее значение дохода
for age_group in data['age_group'].unique():
    for income_type in data['income_type'].unique():
        mean = data.loc[(data['age_group'] == age_group) & (data['income_type'] == income_type), 'total_income'].mean()
        data.loc[(data['total_income'].isna()) & (data['age_group'] == age_group) & (data['income_type'] == income_type), 'total_income'] = mean
        #print(age_group, income_type, mean)

In [39]:
#определим количесвто пропусков после преобразований
print('Пропуски после:', data['total_income'].isna().sum())

Пропуски после: 1


In [40]:
#вспоминаем, что у нас есть проблемный предприниматель
data.loc[(data['age_group'] == 5) & (data['total_income'].isna()), 'total_income'] = data.loc[data['age_group'] == 5, 'total_income'].mean()
data.loc[data['income_type'] == 'предприниматель']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
5936,0,6639.757404,58,высшее,0,женат / замужем,0,M,предприниматель,0,164878.574686,покупка жилой недвижимости,5
18697,0,520.848083,27,высшее,0,гражданский брак,1,F,предприниматель,0,499163.144947,на проведение свадьбы,3


In [41]:
#определим количесвто пропусков после преобразований еще раз и посмотрим инфо по датафрейму
print('Пропуски после:', data['total_income'].isna().sum())
print()
print(data.info())

Пропуски после: 0

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
age_group           21525 non-null int64
dtypes: float64(2), int64(6), object(5)
memory usage: 2.1+ MB
None


## Обработка пропусков: выводы <a id='header_2_3'></a>

В данных наблюдаются странные значения: 
- отрицательное количество детей 
- слишком большое количество детей (по 20 детей у 76 люей выгляди подозрительно)
- XNA в поле 'gender'
- для 101 человека возраст указан '0'

Это могут быть ошибки ввода данных (дети); ошибки отсутствия данных, если человек не предоставил их (в случае пола, возраста).

Также есть дубли. Но только для тех строк, в которых есть NaN значения `days_employed`, `total_income`

#### Пропуски `days_employed`
В данных 10% пропусков. Следует сообщить об этом разработчикам, которые делают выгрузку: все ли ок?

Также стоит сообщить разработчикам об ошибках: отрицательные значения, слишком большие значения.

Заменили пропущенные значения средними значениями внутри категории возраст-тип_занятности. Это оказался не самый надежный способ: для одной записи по этому алгоритму не удалось найти среднее, пришлось делать дополнительные расчеты. Но одна запись — не боль.

Вопросы к данным: 
- что такое стаж у безработного, в декрете? 


#### Пропуски `total_income`
В данных 10% пропусков. Следует сообщить об этом разработчикам, которые делают выгрузку: все ли ок?

Заменили пропущенные значения средними значениями внутри категории возраст-тип_занятности. Это оказался не самый надежный способ: для одной записи по этому алгоритму не удалось найти среднее, пришлось делать дополнительные расчеты. Но одна запись — не боль.


## Замена типов данных <a id = 'header_2_4'></a>

In [42]:
data['days_employed'] = data['days_employed'].astype('int')
print(data.info())
data.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
age_group           21525 non-null int64
dtypes: float64(1), int64(7), object(5)
memory usage: 2.1+ MB
None


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,4
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,4
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,3
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,3
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,5
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,3
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,4
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,5
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,3
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,4


## Замена типов данных: вывод <a id ='header_2_5'></a>

Заменили *float64* на *int64* для поля __days_employed__. Также можно заменить на целочисленный тип значения в поле __total_income__, но деньги лучше оставлять во *float*, чтобы не терять в точности.

Применен метод *astype()*, так как он позволяет выбирать тип данных, тогда как *to_numeric()* переводит только *str* во *float*.

<font color='brown'>Я все же считаю, копейки в уровнях дохода нас не интересуют, так что тоже можно заменить на int. Это сэкономит место и время на обработку, что может быть критично, если данных много. 

## Обработка дубликатов <a id = 'header_2_6'></a>

In [43]:
print('Количество дубликатов до:', data.duplicated().sum())

Количество дубликатов до: 71


In [44]:
data = data.drop_duplicates().reset_index(drop = True)
print('Количество дубликатов после:', data.duplicated().sum())

Количество дубликатов после: 0


## Обработка дубликатов: вывод <a href='header_2_7'></a>

Для удаления дубликатов применили метод _drop_duplicates()_ вместе с методом *reset_index()*, чтобы не создавать столбец со старыми значениями индексов. 
Важно было привести в единому регистру строковые значения. Иначе могли не отловить некоторые из дубликатов.

Причины появления дубликатов: 

- ошибка загрузки данных
- на самом деле это не дубликаты. Это возможно, так как для всех строк-дублей изначально не были указаны *days_employed*, *total_income*. В данных нет id пользователя и нет дат/флагов, по которым можно сказать о том, что это неактивные/исторические записи


## Лемматизация <a id = 'header_2_8'></a>

In [57]:
!pip install pymystem3
from pymystem3 import Mystem
m = Mystem()
lemmas = m.lemmatize(' '.join(data['purpose']))
from collections import Counter
print(Counter(lemmas))

Counter({' ': 55023, 'недвижимость': 6351, 'покупка': 5897, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2324, 'свой': 2230, 'на': 2222, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'подержать': 853, 'проведение': 768, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'приобретение': 461, 'профильный': 436, 'подержанный': 111, '\n': 1})


In [46]:
data['purpose'].value_counts().to_frame()

,purpose
свадьба,791
на проведение свадьбы,768
сыграть свадьбу,765
операции с недвижимостью,675
покупка коммерческой недвижимости,661
операции с жильем,652
покупка жилья для сдачи,651
операции с коммерческой недвижимостью,650
жилье,646
покупка жилья,646


In [47]:
def corr_purpose(purpose):
    if 'недвижимость' in m.lemmatize(purpose):
        return 'недвижимость'
    if 'жилье' in m.lemmatize(purpose):
        return 'жилье'
    if 'автомобиль' in m.lemmatize(purpose):
        return 'автомобиль'
    if 'образование' in m.lemmatize(purpose):
        return 'образование'
    if 'свадьба' in m.lemmatize(purpose):
        return 'свадьба'
    return 'другое'

def corr_purpose_id(purpose):
    if purpose == 'недвижимость':
        return 1
    if purpose == 'жилье':
        return 2
    if purpose == 'автомобиль':
        return 3
    if purpose == 'образование':
        return 4
    if purpose == 'свадьба':
        return 5
    return 0

data['purpose_corr'] = data['purpose'].apply(corr_purpose)
data['purpose_id'] = data['purpose_corr'].apply(corr_purpose_id)
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,purpose_corr,purpose_id
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,4,жилье,2
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,4,автомобиль,3
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,3,жилье,2
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,3,образование,4
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,5,свадьба,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем,4,жилье,2
21450,0,14330,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем,6,автомобиль,3
21451,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость,4,недвижимость,1
21452,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля,4,автомобиль,3


## Лемматизация: вывод <a id = 'header_2_9'></a>

Для лемматизации применили метод из библиотеки **pymystem3**.  Чтобы применить метод, использовали join: он объединил все значения из столбца **purpose** в строку, а *lemmatize* привел слова к леммам. С помощью *Counter* из модуля __collections__ подсчитали, как часто встречается каждая лемма.

Самые часто встречающиеся слова в целях кредитования: 

- недвижимость
- покупка
- жилье
- автомобиль
- образование
- операция
- свадьба

При этом стоит учесть, что слова "покупка", "операция" — не самостоятельное, а всегда идут в связке с другим словом (жилье, недвижимость, автомобиль). 

Поэтому итоговый список: 

- недвижимость
- жилье
- автомобиль
- образование
- свадьба

Создали словарь на основе лемматизации. Функция *corr_purpose* присваивает строке обобщенную категорию цели, функция *corr_purpose_id* задает этой категории id'шник. 

## Категоризация данных <a id = 'header_2_10'></a>

In [48]:
data.pivot_table(index = ['education'], values = 'education_id').sort_values(by = 'education_id')

,education_id
education,
высшее,0
среднее,1
неоконченное высшее,2
начальное,3
ученая степень,4


In [49]:
data.pivot_table(index = ['family_status'], values = 'family_status_id').sort_values(by = 'family_status_id')

,family_status_id
family_status,
женат / замужем,0
гражданский брак,1
вдовец / вдова,2
в разводе,3
Не женат / не замужем,4


In [50]:
data.pivot_table(index = ['purpose_corr'], values = 'purpose_id').sort_values(by = 'purpose_id')

,purpose_id
purpose_corr,
недвижимость,1
жилье,2
автомобиль,3
образование,4
свадьба,5


## Категоризация данных: вывод <a id = 'header_2_11'></a>

Из набора данных выделили 3 словаря: уровень образования, семейное положение и цели кредитования (созданный нами). Выделили именно их, так как в данных категориальным значениям явно потставлены в соответствие численные обозначения. Эти словари можно выделить в отдельный файл, оставив в исходном наборе данных тольео численные id вместо строковых значений. С таким представлением будет проще работать.   

# Анализ данных <a id = 'header_3'></a>

## Зависимость между наличием детей и возвратом кредита в срок <a id = 'header_3_1'></a>

In [51]:
report_children = data.groupby(['children']).agg({'debt':['mean', 'count', 'sum']})
report_children.columns = ['%невозврата', 'кол-во заемщиков', 'кол-во должников']
#report_children = report_children.sort_values(by = '%невозврата', ascending = True)
report_children.style.format({'%невозврата':'{:.2%}'})

,%невозврата,кол-во заемщиков,кол-во должников
children,,,
-1,2.13%,47,1
0,7.54%,14091,1063
1,9.23%,4808,444
2,9.45%,2052,194
3,8.18%,330,27
4,9.76%,41,4
5,0.00%,9,0
20,10.53%,76,8


Ошибочные данные (отрицательное количество детей, 20 детей) не рассматриваем. Четко прослеживаемой зависимости нет. Но можно выделить факт, что отсутствие детей или три ребенка могут положительно влиять на возврат кредита в срок.

## Зависимость между семейным положением и возвратом кредита в срок <a id = 'header_3_2'>

In [52]:
report_family_status = data.pivot_table(index = 'family_status', values = 'debt', aggfunc = ['mean', 'count', 'sum'])
report_family_status.columns = ['%невозврата', 'кол-во заемщиков', 'кол-во должников']
report_family_status = report_family_status.sort_values(by = '%невозврата', ascending = False)
report_family_status.style.format({'%невозврата':'{:.2%}'})

,%невозврата,кол-во заемщиков,кол-во должников
family_status,,,
Не женат / не замужем,9.75%,2810,274
гражданский брак,9.35%,4151,388
женат / замужем,7.55%,12339,931
в разводе,7.11%,1195,85
вдовец / вдова,6.57%,959,63


Зависимость есть. Люди с семейным положением "после брака" (вдовец/вдова/в разводе) реже оказываются должниками. Высокий риск для тех, кто холост или не оформил отношения официально. 

## Зависимость между уровнем дохода и возвратом кредита в срок <a id = 'header_3_3'>

In [53]:
data['total_income'].describe()['75%']

200223.31751398585

In [54]:
def income_group (income):
    desc = data['total_income'].describe()
    if income < desc['25%']:#30000:
        return "до {:.2f}".format(desc['25%'])
    if income < desc['50%']:
        return "до {:.2f}".format(desc['50%'])
    if income < desc['75%']:
        return "до {:.2f}".format(desc['75%'])
    return "более {:.2f}".format(desc['75%'])

data['income_group'] = data['total_income'].apply(income_group)
report_income = data.pivot_table(index = 'income_group', values = 'debt', aggfunc = ['mean', 'count', 'sum'])
report_income.columns = ['%невозврата', 'кол-во заемщиков', 'кол-во должников']
report_income = report_income.sort_values(by = '%невозврата', ascending = False)
report_income.style.format({'%невозврата':'{:.2%}'})

,%невозврата,кол-во заемщиков,кол-во должников
income_group,,,
до 151209.94,8.84%,5363,474
до 200223.32,8.73%,5363,468
до 107585.13,7.96%,5364,427
более 200223.32,6.94%,5364,372


Да, есть зависимость. В центральных категориях она не очень проявляется (возможно, необходима иная разбивка). На границе разница заментная. Люди с низким доходом чаще ноказываются должниками, чем люди с высоким доходом.

## Влияние цели кредита на возврат в срок <a id = 'header_3_4'>

In [55]:
report_purpose = data.pivot_table(index = 'purpose_corr', values = 'debt', aggfunc = ['mean', 'count', 'sum'])
report_purpose.columns = ['%невозврата', 'кол-во заемщиков', 'кол-во должников']
report_purpose = report_purpose.sort_values(by = '%невозврата', ascending = False)
report_purpose.style.format({'%невозврата':'{:.2%}'})

,%невозврата,кол-во заемщиков,кол-во должников
purpose_corr,,,
автомобиль,9.36%,4306,403
образование,9.22%,4013,370
свадьба,8.00%,2324,186
недвижимость,7.46%,6351,474
жилье,6.91%,4460,308


Кредит за недвижимость и жилье возвращают более охотно, чем за автомобили и (удивительно) за образование. Свадьбы серединка на половинку, как повезет.

# Общий вывод <a id = 'header_4'>

#### Влияет ли колиество детей и семейное положение на факт погашения кредита в срок.
На вывод влияет факт, что в данных есть некорректные значения. 

Рассмотрим эти два признака вместе: возврат в срок происходит тем реже, чем больше у заемщика детей. Есть отклонения, которые не подходят под эту формулу:
 
- для гражданского брака отсутстиве детей или два ребенка улучшают показатель возврата кредита, а вот один ребенок — ухудшает.
- для замужних три ребенка улучшают показатель возврата в срок.

Кроме того:
- зависимость между семейным положением и возвратом в срок есть. Разведенные и вдовцы самые надежные, следом идут женатые/замужние. Самые ненадежные: холостые или люди, состоящие в гражданском браке. 

В любом случае нужно уточнить данные и перепроверить. Возможно, звапросить более объемную выгрузку. 



In [56]:
report_children = data.groupby(['family_status', 'children']).agg({'debt':['mean', 'count', 'sum']})
report_children.columns = ['%невозврата', 'кол-во заемщиков', 'кол-во должников']
#report_children = report_children.sort_values(by = '%невозврата', ascending = True)
report_children.style.format({'%невозврата':'{:.2%}'})